In [1]:
import sys

sys.path.append("../")

from loaders.LHCO import LhcoRnDLoader
from analysis.scalar import HLFAnalysis
from models.nn import PaeBuilder
from utils import load_json, dump_json

import tensorflow as tf
import numpy as np

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
import glob

configs_folder = '../configs/analysis'
config_files = glob.glob(configs_folder+'/*.json')
config_files = ['../configs/configs/c3d4.json']

In [4]:
config_files[0].replace('.', '/').split('/')[-2]

'c3d4'

In [5]:
import os

plots_dir = './figures/'
if not os.path.exists(plots_dir):
    os.mkdir(plots_dir)
results_history = {
 'id': [],
 'config': [],
 'js_div_train': [],
 'js_div_test': [],
 'sig_eff': [],
 'bkg_rej': [],
 'auc': []}

In [6]:
 for file in config_files:
    analysis_cfg = load_json(file)
    analysis_cfg

    analysis_cfg["NF:n_dims"] = analysis_cfg["AE:encoding_dim"]
    config = analysis_cfg.copy()

    loader_json = analysis_cfg.pop('ANA:loader')
    dataset_json = analysis_cfg.pop('ANA:dataset')
    density_estimator = analysis_cfg.pop('ANA:estimator')
    prc = analysis_cfg.pop('ANA:percentile')

    loader = LhcoRnDLoader.from_json(loader_json)
    dataset_cfg = load_json(dataset_json)
    dataset = loader.make_full_dataset(**dataset_cfg)

    builder = PaeBuilder()

    pae, ae_train, nf_train = builder.from_json(analysis_cfg)
    task = HLFAnalysis(pae, dataset=dataset)
    task.reweighting(estimator=density_estimator, fit_key='mjj_train')
    if 'cond' in config['nf_model']:
        task.make_cond_inputs(['mjj_train', 'mjj_test', 'mjj_valid'])
    task.train(ae_train,nf_train)
    result = task.evaluate(prc = prc)
    for key in result.keys():
        results_history[key].append(result[key])
    
    id = config_files[0].replace('.', '/').split('/')[-2]
    results_history['id'].append(id)
    results_history['config'].append(config)
    task.plot_training(plots_dir+id+'_train.png')
    task.plot_latent_space(plots_dir+id+'_latent.png')



[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
800/800 [==============================] - 9s 9ms/step - loss: -15.4158 - val_loss: -24.8004


In [7]:
results_history

{'id': ['c3d4'],
 'config': [{'ANA:loader': '../configs/loader/default_rnd.json',
   'ANA:dataset': '../configs/loader/default_dataset.json',
   'ANA:estimator': 'gmm',
   'ANA:percentile': 95,
   'ae_model': 'dense_ae',
   'nf_model': 'maf',
   'AE:input_dim': 47,
   'AE:encoding_dim': 12,
   'AE:units': [30, 20, 15],
   'AE:hidden_activation': 'relu',
   'AE:output_activation': 'sigmoid',
   'AE:weight_reg': {'l1': 1e-06},
   'AE:bias_reg': {},
   'AE:output_reg': {},
   'NF:n_dims': 12,
   'NF:n_layers': 5,
   'NF:units': [32, 32, 32, 32],
   'ae_optimizer': 'adam',
   'nf_optimizer': 'adam',
   'ae_optimizer_kwargs': {'lr': 0.001},
   'nf_optimizer_kwargs': {'lr': 0.005},
   'ae_callbacks': ['reduce_lr_on_plateau'],
   'ae_callbacks_kwargs': [{'factor': 0.2, 'patience': 10, 'verbose': 1}],
   'nf_callbacks': ['reduce_lr_on_plateau'],
   'nf_callbacks_kwargs': [{'factor': 0.2, 'patience': 5, 'verbose': 1}],
   'ae_epochs': 1,
   'ae_batch_size': 200,
   'nf_epochs': 1,
   'nf_batch_

# Plotting results

In [1]:
import sys

sys.path.append("../")

from utils import load_json, dump_json

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots

import pandas as pd

from plotly.offline import download_plotlyjs, init_notebook_mode
init_notebook_mode(connected = True)
pio.templates.default = "plotly_dark"

In [3]:
results = load_json("./regularization_result.json")

In [4]:
results.keys()

dict_keys(['id', 'config', 'js_div_train', 'js_div_test', 'sig_eff', 'bkg_rej', 'auc'])

In [5]:
cfgs = results.pop('config')

In [6]:
relevant_keys = ['ANA:estimator', 'nf_model', 'AE:encoding_dim', 'AE:weight_reg']
cfgs_merged = {}
for d in cfgs:
    for k in relevant_keys: 
        cfgs_merged.setdefault(k, []).append(d[k])

In [7]:
cfgs_merged['l1'] = [x['l1'] for x in cfgs_merged['AE:weight_reg']]
cfgs_merged['l2'] = [x['l2'] for x in cfgs_merged['AE:weight_reg']]

In [8]:
data = {**results, **cfgs_merged}
data['encoding_dim'] = data.pop('AE:encoding_dim')
data['js_div'] = data.pop('js_div_train')
data.pop('AE:weight_reg')
data['density_estimator'] = data.pop('ANA:estimator')
data['reg_l1_l2'] =['' for x in data['density_estimator']]

df = pd.DataFrame(data)
df['reg_l1_l2'][(df['l1'] == 1e-5) & (df['l2'] == 1e-5)] = '1e-5/1e-5'
df['reg_l1_l2'][(df['l1'] == 1e-6) & (df['l2'] == 1e-5)] = '1e-6/1e-5'
df['reg_l1_l2'][(df['l1'] == 1e-6) & (df['l2'] == 1e-6)] = '1e-6/1e-6'
df['reg_l1_l2'][(df['l1'] == 1e-5) & (df['l2'] == 1e-6)] = '1e-5/1e-6'

In [9]:

fig = px.scatter(df, y='auc', x='encoding_dim', size='js_div', color='reg_l1_l2', hover_data=['auc', 'js_div', 'l1', 'l2'])
fig.update_layout(title_text='Initial Hyperparameter Scan')
fig.write_html('results.html')
fig.show('vscode')

In [11]:
print(len(data['l1']))

64
